# Nomorebored project - draft.

### Connect AEMET API

In [11]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv

# Cargar las claves API desde el archivo .env
load_dotenv()

# API key de AEMET
AEMET_API_KEY = os.getenv('AEMET_API_KEY')

# URL para obtener la lista de municipios desde AEMET
url_municipios = "https://opendata.aemet.es/opendata/api/maestro/municipios"

# Función para obtener la lista de municipios y códigos desde AEMET
def obtener_lista_municipios():
    headers = {
        'api_key': AEMET_API_KEY
    }

    response = requests.get(url_municipios, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        if 'datos' in data:
            # Obtener la URL para los datos de municipios
            url_datos = data['datos']
            response_datos = requests.get(url_datos)

            if response_datos.status_code == 200:
                municipios = response_datos.json()
                return municipios
            else:
                print(f"Error al obtener los datos de municipios: {response_datos.status_code}")
                return None
        else:
            print("Error: la clave 'datos' no está presente en la respuesta.")
            return None
    else:
        print(f"Error al obtener la lista de municipios: {response.status_code}")
        return None

# Función para guardar la lista de municipios en un archivo CSV usando pandas
def guardar_municipios_csv(municipios, nombre_archivo='municipios_aemet.csv'):
    # Crear un DataFrame temporal con los municipios
    df = pd.DataFrame(municipios)

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(nombre_archivo, index=False, encoding='utf-8')
    print(f"Lista de municipios guardada en {nombre_archivo}")

# Función para buscar el código de un municipio a partir de su nombre (ya obtenido de Google)
def buscar_codigo_municipio_por_nombre(municipios, nombre_municipio):
    for municipio in municipios:
        if municipio['nombre'].lower() == nombre_municipio.lower():
            return municipio['id']
    return None

# Ejecutar la función para obtener y guardar municipios
if __name__ == '__main__':
    # Obtener la lista de municipios desde la API de AEMET
    municipios = obtener_lista_municipios()

    if municipios:
        # Guardar la lista completa de municipios en un archivo CSV usando pandas
        guardar_municipios_csv(municipios)
    else:
        print("No se pudo obtener la lista de municipios.")

Lista de municipios guardada en municipios_aemet.csv


In [12]:
# Cargar el archivo CSV de municipios en un DataFrame
df_municipios = pd.read_csv('municipios_aemet.csv')
df_municipios.head()

,latitud,id_old,url,latitud_dec,altitud,capital,num_hab,zona_comarcal,destacada,nombre,longitud_dec,id,longitud
0,"40º32'54.450744""",44004.0,ababuj-id44001,40.548459,1372,Ababuj,65,624401,0,Ababuj,-0.807801,id44001,"-0º48'28.084212"""
1,"40º54'58.824504""",40004.0,abades-id40001,40.916340,971,Abades,873,674001,0,Abades,-4.267874,id40001,"-4º16'4.346004"""
2,"43º8'51.525564""",48010.0,abadino-abadino-zelaieta-id48001,43.147646,144,Abadiño-Zelaieta,7504,754802,0,Abadiño,-2.606873,id48001,"-2º36'24.743484"""
3,"40º15'34.315272""",10004.0,abadia-id10001,40.259532,451,Abadía,324,701001,0,Abadía,-5.977858,id10001,"-5º58'40.289016"""
4,"43º21'46.874736""",27010.0,abadin-abadin-o-provecende-id27001,43.363021,515,Abadín o Provecende,2646,712702,0,Abadín,-7.472145,id27001,"-7º28'19.72182"""


In [13]:
import requests
import os
from dotenv import load_dotenv

# Cargar las variables del archivo .env
load_dotenv()

# Obtener la API key desde el archivo .env
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Definir la función para obtener la geolocalización (latitud y longitud)
def obtener_geolocalizacion():
    # URL para la API de Google Geolocation
    url = f"https://www.googleapis.com/geolocation/v1/geolocate?key={GOOGLE_API_KEY}"
    
    # Si usas WiFi o quieres hacer la solicitud desde la red
    data = {
        "considerIp": True
    }
    
    response = requests.post(url, json=data)
    if response.status_code == 200:
        data = response.json()
        latitud = data['location']['lat']
        longitud = data['location']['lng']
        return latitud, longitud
    else:
        print(f"Error en la solicitud de geolocalización: {response.status_code}")
        return None

# Función para convertir latitud y longitud a municipio usando Google Geocoding API
def obtener_municipio(latitud, longitud):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitud},{longitud}&key={GOOGLE_API_KEY}"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data['results']) > 0:
            address_components = data['results'][0]['address_components']
            
            municipio_locality = None
            municipio_alternative = None
            
            # 1. Intentar obtener el 'locality' primero (que debería ser Chiclana de la Frontera)
            for component in address_components:
                if 'locality' in component['types']:
                    municipio_locality = component['long_name']
                    
                    # Verificar si el municipio está en el CSV
                    if obtener_codigo_municipio(municipio_locality) is not None:
                        return municipio_locality
            
            # 2. Si no encontramos el 'locality', buscar en 'administrative_area_level_4' o superior (evitar sublocalidades como Pago del Humo)
            for result in data['results']:
                for component in result['address_components']:
                    if 'locality' in component['types'] or 'administrative_area_level_4' in component['types']:
                        municipio_alternative = component['long_name']
                        
                        # Verificar si el municipio está en el CSV
                        if obtener_codigo_municipio(municipio_alternative) is not None:
                            return municipio_alternative
            
            # 3. Si no encontramos nada, intentamos con 'administrative_area_level_3'
            for result in data['results']:
                for component in result['address_components']:
                    if 'administrative_area_level_3' in component['types']:
                        municipio_alternative = component['long_name']
                        
                        # Verificar si está en el CSV
                        if obtener_codigo_municipio(municipio_alternative) is not None:
                            return municipio_alternative
            
        # Si no encontramos nada relevante, devolvemos None
        return None
    else:
        print(f"Error en la solicitud de Google Geocoding: {response.status_code}")
        return None

# Función para obtener el código del municipio desde el CSV
def obtener_codigo_municipio(municipio_nombre):
    municipio_fila = df_municipios[df_municipios['nombre'].str.lower() == municipio_nombre.lower()]
    
    if not municipio_fila.empty:
        codigo_municipio = municipio_fila.iloc[0]['id']
        return codigo_municipio
    else:
        return None

# Ejecución del script
if __name__ == '__main__':
    # Obtener geolocalización
    ubicacion = obtener_geolocalizacion()
    if ubicacion:
        lat, lon = ubicacion
        print(f"Ubicación detectada: latitud {lat}, longitud {lon}")

        # Obtener el municipio basado en la geolocalización
        municipio = obtener_municipio(lat, lon)
        if municipio:
            print(f"Municipio detectado: {municipio}")

            # Obtener el código del municipio desde el CSV y filtrar solo el número eliminando el "id" para la posterior obtención del clima
            codigo_municipio = obtener_codigo_municipio(municipio)[2:]
            print(f"Código del municipio: {codigo_municipio}")


Ubicación detectada: latitud 36.3896325, longitud -6.0891032
Municipio detectado: Chiclana de la Frontera
Código del municipio: 11015


In [15]:
import requests
import os
from dotenv import load_dotenv
from collections import Counter

# Cargar las claves API desde el archivo .env
load_dotenv()

# Extraer la API key desde el .env
AEMET_API_KEY = os.getenv('AEMET_API_KEY')

def obtener_prediccion_aemet(codigo_municipio):
    url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/diaria/{codigo_municipio}"
    headers = {
        'api_key': AEMET_API_KEY
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if 'datos' in data:
            datos_url = data['datos']
            datos_response = requests.get(datos_url)
            if datos_response.status_code == 200:
                clima_data = datos_response.json()
                return clima_data
            else:
                print(f"Error al obtener los datos del clima: {datos_response.status_code}")
        else:
            print(f"Error: no se encontró la clave 'datos' en la respuesta. Detalles: {data}")
    else:
        print(f"Error en la solicitud a AEMET: {response.status_code}")

def obtener_estado_cielo_mas_frecuente(prediccion_hoy):
    # Lista para almacenar los estados del cielo durante el día
    estados_cielo = []

    # Recorrer todos los periodos del día y extraer el estado del cielo
    for periodo in prediccion_hoy['estadoCielo']:
        descripcion = periodo.get('descripcion', '')
        if descripcion:  # Solo agregar si hay una descripción válida
            estados_cielo.append(descripcion)
    
    if estados_cielo:
        # Contar la frecuencia de cada descripción y elegir la más común
        contador_estados = Counter(estados_cielo)
        estado_mas_comun = contador_estados.most_common(1)[0][0]
        return estado_mas_comun
    else:
        return 'Información no disponible'

def procesar_datos_clima(clima_data):
    if clima_data:
        # Obtener el primer día de predicción
        prediccion_hoy = clima_data[0]['prediccion']['dia'][0]
        
        # Extraer temperaturas
        temp_maxima = prediccion_hoy['temperatura']['maxima']
        temp_minima = prediccion_hoy['temperatura']['minima']
        
        # Obtener el estado del cielo más frecuente del día
        estado_cielo = obtener_estado_cielo_mas_frecuente(prediccion_hoy)
        
        # Mostrar resultados
        print(f"Estás en {municipio}.")
        print(f"Temperatura máxima: {temp_maxima}°C")
        print(f"Temperatura mínima: {temp_minima}°C")
        print(f"Estado del cielo: {estado_cielo}")

# Código para el municipio de ejemplo
clima = obtener_prediccion_aemet(codigo_municipio)

if clima:
    procesar_datos_clima(clima)
else:
    print("No se pudo obtener el clima.")

Estás en Chiclana de la Frontera.
Temperatura máxima: 25°C
Temperatura mínima: 19°C
Estado del cielo: Poco nuboso


In [16]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime

# Cargar las claves API desde el archivo .env
load_dotenv()

# Extraer la API key desde el .env
AEMET_API_KEY = os.getenv('AEMET_API_KEY')

def obtener_prediccion_aemet(codigo_municipio):
    url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/diaria/{codigo_municipio}"
    headers = {
        'api_key': AEMET_API_KEY
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if 'datos' in data:
            datos_url = data['datos']
            datos_response = requests.get(datos_url)
            if datos_response.status_code == 200:
                clima_data = datos_response.json()
                return clima_data
            else:
                print(f"Error al obtener los datos del clima: {datos_response.status_code}")
        else:
            print(f"Error: no se encontró la clave 'datos' en la respuesta. Detalles: {data}")
    else:
        print(f"Error en la solicitud a AEMET: {response.status_code}")

def obtener_bloque_tiempo(hora_actual):
    # Bloques de tiempo en los datos de la API
    bloques = [
        (0, 6),
        (6, 12),
        (12, 18),
        (18, 24)
    ]
    for inicio, fin in bloques:
        if inicio <= hora_actual < fin:
            return f"{inicio:02d}-{fin:02d}"
    return None

def obtener_estado_cielo_por_bloque(prediccion_hoy, bloque):
    for periodo in prediccion_hoy['estadoCielo']:
        if periodo['periodo'] == bloque:
            return periodo.get('descripcion', 'Información no disponible')
    return 'Información no disponible'

def obtener_viento_por_bloque(prediccion_hoy, bloque):
    for viento in prediccion_hoy['viento']:
        if viento['periodo'] == bloque:
            return viento.get('velocidad', 'Información no disponible')
    return 'Información no disponible'

def obtener_lluvia_por_bloque(prediccion_hoy, bloque):
    for precipitacion in prediccion_hoy['probPrecipitacion']:
        if precipitacion['periodo'] == bloque:
            return precipitacion.get('value', 'Información no disponible')
    return 'Información no disponible'

def procesar_datos_clima(clima_data, municipio):
    if clima_data:
        # Obtener el primer día de predicción
        prediccion_hoy = clima_data[0]['prediccion']['dia'][0]

        # Obtener la hora actual y el bloque correspondiente
        hora_actual = datetime.now().hour
        bloque = obtener_bloque_tiempo(hora_actual)
        
        # Obtener el estado del cielo, viento y lluvia para el bloque actual
        estado_cielo_actual = obtener_estado_cielo_por_bloque(prediccion_hoy, bloque)
        viento_actual = obtener_viento_por_bloque(prediccion_hoy, bloque)
        lluvia_actual = obtener_lluvia_por_bloque(prediccion_hoy, bloque)

        # Mostrar resultados
        hora_actual_str = datetime.now().strftime("%H:%M")
        print(f"Estás en {municipio}.")
        print(f"Hora actual: {hora_actual_str}")
        print(f"Estado del cielo: {estado_cielo_actual}")
        print(f"Velocidad del viento: {viento_actual} km/h")
        print(f"Probabilidad de lluvia: {lluvia_actual}%")

clima = obtener_prediccion_aemet(codigo_municipio)

if clima:
    procesar_datos_clima(clima, municipio)
else:
    print("No se pudo obtener el clima.")


Estás en Chiclana de la Frontera.
Hora actual: 12:02
Estado del cielo: Poco nuboso
Velocidad del viento: 30 km/h
Probabilidad de lluvia: 0%


In [17]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime

# Cargar las claves API desde el archivo .env
load_dotenv()

# Extraer la API key desde el .env
AEMET_API_KEY = os.getenv('AEMET_API_KEY')

# Función para obtener la predicción horaria de AEMET
def obtener_prediccion_horaria_aemet(codigo_municipio):
    url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/horaria/{codigo_municipio}"
    headers = {
        'api_key': AEMET_API_KEY
    }

    # Realizar la solicitud a la API de AEMET
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if 'datos' in data:
            # Obtener el enlace con los datos de la predicción horaria
            datos_url = data['datos']
            datos_response = requests.get(datos_url)
            if datos_response.status_code == 200:
                clima_data = datos_response.json()
                return clima_data
            else:
                print(f"Error al obtener los datos de predicción horaria: {datos_response.status_code}")
        else:
            print(f"Error: no se encontró la clave 'datos' en la respuesta. Detalles: {data}")
    else:
        print(f"Error en la solicitud a AEMET: {response.status_code}")

# Función para obtener la hora actual redondeada a la hora más cercana
def obtener_hora_actual():
    return datetime.now().hour

# Función para obtener la temperatura de la hora actual
def obtener_temperatura_actual(prediccion_hoy):
    hora_actual = obtener_hora_actual()  # Obtener la hora actual

    # Recorrer todos los datos de temperatura disponibles
    for temp_dato in prediccion_hoy['temperatura']:
        if int(temp_dato['periodo']) == hora_actual:  # Comparar con la hora actual exacta
            return temp_dato['value']

    return 'Información no disponible'

def procesar_temperatura(clima_data):
    if clima_data:
        # Obtener el primer día de predicción
        prediccion_hoy = clima_data[0]['prediccion']['dia'][0]
        
        # Obtener la temperatura actual basada en la hora más cercana
        temp_actual = obtener_temperatura_actual(prediccion_hoy)

        # Mostrar resultados
        hora_actual = datetime.now().strftime("%H:%M")
        print(f"Hora actual: {hora_actual}")
        print(f"Temperatura actual: {temp_actual}°C")

# Obtener y mostrar el JSON de predicción horaria
clima = obtener_prediccion_horaria_aemet(codigo_municipio)

if clima:
    procesar_temperatura(clima)
else:
    print("No se pudo obtener el clima.")


Hora actual: 12:02
Temperatura actual: 22°C


In [ ]:
import pandas as pd
import numpy as np
import random
from itertools import product

# Definir estructura de categorías y subcategorías con sus plantillas
category_templates = {
    'Deporte': {
        'Fitness': [
            'Entrenamiento HIIT de {tiempo} minutos',
            'Rutina de pesas para {objetivo}',
            'Sesión de cardio {intensidad}',
            'Entrenamiento funcional {nivel}',
            'Clase de aeróbics {estilo}',
            'Boot camp {intensidad}',
            'Circuito de ejercicios {tipo}',
            'Entrenamiento con kettlebells',
            'TRX {nivel}',
            'CrossFit WOD del día'
        ],
        'Deportes de equipo': [
            'Partido amistoso de {deporte}',
            'Entrenamiento de {deporte} para {nivel}',
            'Práctica de técnicas de {deporte}',
            'Torneo informal de {deporte}',
            'Clase de iniciación a {deporte}',
            'Juego en equipo de {deporte}',
            'Mini torneo de {deporte}',
            'Sesión táctica de {deporte}',
            'Partido de {deporte} en {lugar}',
            'Práctica de {deporte} recreativo'
        ]
    },
    'Cultura': {
        'Museos': [
            'Visita al museo de {tipo}',
            'Exposición temporal de {tema}',
            'Tour guiado por {museo}',
            'Visita virtual al museo {nombre}',
            'Taller interactivo en {museo}',
            'Exposición de {artista}',
            'Recorrido temático por {museo}',
            'Visita nocturna a {museo}',
            'Exhibición especial de {tema}',
            'Tour privado por {museo}'
        ],
        'Teatro': [
            'Obra de teatro {género}',
            'Monólogo de {temática}',
            'Musical {título}',
            'Improvisación teatral',
            'Teatro experimental {estilo}',
            'Microteatro {temática}',
            'Performance artística',
            'Teatro infantil {tema}',
            'Comedia en vivo',
            'Drama contemporáneo'
        ]
    },
    'Gastronomía': {
        'Cocina internacional': [
            'Taller de cocina {país}',
            'Clase de {plato} tradicional',
            'Degustación de {cocina}',
            'Preparación de {plato} típico',
            'Curso de cocina {estilo}',
            'Masterclass de {plato}',
            'Demostración de cocina {país}',
            'Cena temática {país}',
            'Workshop de {técnica} culinaria',
            'Experiencia gastronómica {país}'
        ]
    }
}

# Datos para rellenar las plantillas
template_data = {
    'tiempo': ['15', '20', '30', '45', '60'],
    'objetivo': ['principiantes', 'tonificación', 'pérdida de peso', 'fuerza', 'resistencia'],
    'intensidad': ['baja', 'media', 'alta', 'adaptativa'],
    'nivel': ['principiante', 'intermedio', 'avanzado'],
    'estilo': ['clásico', 'moderno', 'fusión', 'experimental'],
    'tipo': ['cuerpo completo', 'core', 'tren superior', 'tren inferior'],
    'deporte': ['fútbol', 'baloncesto', 'voleibol', 'balonmano', 'hockey'],
    'lugar': ['parque local', 'polideportivo', 'centro deportivo', 'playa'],
    'tipo': ['arte contemporáneo', 'historia', 'ciencia', 'tecnología'],
    'tema': ['naturaleza', 'historia local', 'arte moderno', 'innovación'],
    'museo': ['museo nacional', 'museo local', 'galería de arte', 'centro cultural'],
    'nombre': ['Principal', 'Histórico', 'Moderno', 'De la Ciudad'],
    'artista': ['artista local', 'artista internacional', 'artista emergente'],
    'género': ['comedia', 'drama', 'musical', 'experimental'],
    'temática': ['social', 'histórica', 'contemporánea', 'familiar'],
    'título': ['El Fantasma', 'Cats', 'Chicago', 'El Rey León'],
    'país': ['italiana', 'japonesa', 'mexicana', 'india', 'francesa'],
    'plato': ['sushi', 'pasta', 'tacos', 'curry', 'paella'],
    'cocina': ['mediterránea', 'asiática', 'latinoamericana', 'fusión'],
    'técnica': ['tradicional', 'moderna', 'molecular', 'fusión']
}

def generate_unique_activities(n_activities):
    activities = []
    used_names = set()
    
    # Generar todas las combinaciones posibles de categoría y subcategoría
    all_combinations = []
    for category in category_templates:
        for subcategory in category_templates[category]:
            all_combinations.append((category, subcategory))
    
    while len(activities) < n_activities:
        # Seleccionar una combinación aleatoria
        category, subcategory = random.choice(all_combinations)
        
        # Seleccionar una plantilla aleatoria
        template = random.choice(category_templates[category][subcategory])
        
        # Rellenar la plantilla
        activity_name = template
        for key in template_data:
            if '{' + key + '}' in activity_name:
                activity_name = activity_name.replace('{' + key + '}', random.choice(template_data[key]))
        
        # Verificar si el nombre es único
        if activity_name not in used_names:
            used_names.add(activity_name)
            
            indoor_outdoor = 'Indoor' if category in ['Cultura', 'Gastronomía'] else 'Outdoor'
            
            activities.append({
                'ID': len(activities) + 1,
                'Nombre_Tarea': activity_name,
                'Categoria_Principal': category,
                'Subcategoria': subcategory,
                'Indoor_Outdoor': indoor_outdoor,
                'Temp_Min': 18 if indoor_outdoor == 'Indoor' else 15,
                'Temp_Max': 28 if indoor_outdoor == 'Indoor' else 30,
                'Tiempo_Estimado_Minutos': random.choice([30, 45, 60, 90, 120, 180, 240]),
                'Requiere_Ubicacion': subcategory in ['Museos', 'Teatro', 'Cine', 'Cocina internacional'],
                'API_Categoria': 'place_category' if subcategory in ['Museos', 'Teatro', 'Cine'] else None,
                'Descripcion': f"Explora una emocionante {activity_name} dentro de la categoría de {category}"
            })
    
    return activities

# Generar actividades
activities = generate_unique_activities(200)

# Crear DataFrame
df = pd.DataFrame(activities)

# Guardar en diferentes formatos
df.to_csv('no_more_bored_activities_large.csv', index=False)

# Mostrar estadísticas básicas
print("\nDistribución de categorías principales:")
print(df['Categoria_Principal'].value_counts())
print("\nDistribución Indoor/Outdoor:")
print(df['Indoor_Outdoor'].value_counts())
print("\nEjemplos de actividades generadas:")
print(df['Nombre_Tarea'].head(10))